In [61]:
from datetime import datetime

import pandas as pd
import arduino_helpers.hardware.teensy as teensy
from arduino_helpers.hardware.teensy.adc import ADC_DESCRIPTIONS
import arduino_helpers.hardware.arduino as arduino
from arduino_rpc.protobuf import resolve_field_values

from teensy_minimal_rpc import SerialProxy
import teensy_minimal_rpc.ADC as adc
import teensy_minimal_rpc.DMA as dma
import teensy_minimal_rpc.SIM as sim

In [63]:
try:
    del proxy
except NameError:
    pass

proxy = SerialProxy(baudrate=512000)

proxy.pin_mode(13, 1)
proxy.digital_write(13, 1)
proxy.enableDMA(teensy.ADC_0)
proxy.enableInterrupts(teensy.ADC_0)
proxy.dma_start(256)

# Set ADC parameters
proxy.setAveraging(4, teensy.ADC_0)
proxy.setResolution(10, teensy.ADC_0)
proxy.setConversionSpeed(teensy.ADC_VERY_HIGH_SPEED, teensy.ADC_0)
proxy.setSamplingSpeed(teensy.ADC_VERY_HIGH_SPEED, teensy.ADC_0)

In [27]:
tcd = dma.TCD.FromString(proxy.read_dma_TCD(0).tostring())

df_tdc_fields = resolve_field_values(tcd)
df_tdc_fields[['full_name', 'value']].dropna()

,full_name,value
parent_name,,
,DLASTSGA,0
,SLAST,0
,SOFF,0
,DOFF,2
,DADDR,536842240
,SADDR,1073983504
CITER_ELINKNO,CITER_ELINKNO.ELINK,False
CITER_ELINKNO,CITER_ELINKNO.ITER,1
ATTR,ATTR.SMOD,0


In [14]:
scgc6 = sim.R_SCGC6.FromString(proxy.read_sim_SCGC6().tostring())

df_scgc6_fields = resolve_field_values(scgc6)
df_scgc6_fields[['full_name', 'value']].dropna().T

parent_name,,,,,,,,,,,,,,
full_name,USBDCD,SPI0,SPI1,RTC,FTM1,FTM0,I2S,DMAMUX,CRC,FTFL,ADC0,FLEXCAN0,PIT,PDB
value,False,False,False,False,True,True,False,True,False,True,True,False,False,False


In [29]:
scgc7 = sim.R_SCGC7.FromString(proxy.read_sim_SCGC7().tostring())

df_scgc7_fields = resolve_field_values(scgc7)
df_scgc7_fields[['full_name', 'value']].dropna().T

parent_name,
full_name,DMA
value,True


In [43]:
proxy.startSingleRead(teensy.A1, teensy.ADC_0)

1

In [58]:
proxy.adc_read()

array([0, 0], dtype=uint16)

In [65]:
adc0_registers = adc.Registers.FromString(proxy.read_adc_registers(0).tostring())
df_fields = resolve_field_values(adc0_registers).join(ADC_DESCRIPTIONS, on='full_name') 
df_fields.loc[(df_fields.index != '') |
              df_fields.full_name.str.startswith('R'),
              ['full_name', 'value', 'short_description']]

,full_name,value,short_description
parent_name,,,
,RB,0,ADC Data Result Register
,RA,0,ADC Data Result Register
CFG2,CFG2.MUXSEL,A,ADC mux select
CFG2,CFG2.ADLSTS,ADD_6_ADCK_CYCLES,Long sample time select
CFG2,CFG2.ADHSC,True,High-speed configuration
CFG2,CFG2.ADACKEN,False,Asynchronous clock output enable
CFG1,CFG1.ADLSMP,SHORT,Sample time configuration
CFG1,CFG1.ADICLK,BUS_CLOCK_DIV_2,Input clock select
CFG1,CFG1.MODE,_10_BIT,Conversion mode selection


In [24]:
teensy.adc.SC1A_TO_CHANNEL_ADC0[df_fields.set_index('full_name').loc['SC1A.ADCH'].value]
teensy.adc.CHANNEL_TO_SC1A_ADC0[teensy.A0]

5

In [16]:
proxy.analogRead(teensy.A0, teensy.ADC_0)

1023

In [36]:
[(k.full_name, v) for k, v in adc0_registers.ListFields()]

[('teensy._3_1.ADC_REGISTERS.SC1A', <ADC_pb2.R_SC1 at 0xa2b6630>),
 ('teensy._3_1.ADC_REGISTERS.SC1B', <ADC_pb2.R_SC1 at 0xa2b6770>),
 ('teensy._3_1.ADC_REGISTERS.CFG1', <ADC_pb2.R_CFG1 at 0xa2b6670>),
 ('teensy._3_1.ADC_REGISTERS.CFG2', <ADC_pb2.R_CFG2 at 0xa2b66b0>),
 ('teensy._3_1.ADC_REGISTERS.RA', 0),
 ('teensy._3_1.ADC_REGISTERS.RB', 0),
 ('teensy._3_1.ADC_REGISTERS.CV1', 0),
 ('teensy._3_1.ADC_REGISTERS.CV2', 0),
 ('teensy._3_1.ADC_REGISTERS.SC2', <ADC_pb2.R_SC2 at 0xa2b67b0>),
 ('teensy._3_1.ADC_REGISTERS.SC3', <ADC_pb2.R_SC3 at 0xa2b67f0>),
 ('teensy._3_1.ADC_REGISTERS.OFS', 65535),
 ('teensy._3_1.ADC_REGISTERS.PG', 33479),
 ('teensy._3_1.ADC_REGISTERS.MG', 33444),
 ('teensy._3_1.ADC_REGISTERS.CLPD', 18),
 ('teensy._3_1.ADC_REGISTERS.CLPS', 46),
 ('teensy._3_1.ADC_REGISTERS.CLP4', 705),
 ('teensy._3_1.ADC_REGISTERS.CLP3', 356),
 ('teensy._3_1.ADC_REGISTERS.CLP2', 179),
 ('teensy._3_1.ADC_REGISTERS.CLP1', 91),
 ('teensy._3_1.ADC_REGISTERS.CLP0', 45),
 ('teensy._3_1.ADC_REGISTER

In [11]:
adc0_registers.SC3.

True

In [140]:
proxy.start_timer(20)

In [141]:
%matplotlib inline

In [219]:
frames = []
for i in xrange(100):
    result = proxy.adc_read()
    adc_count = result[:2].view('uint32')[0]
    channel_start_i = [0, 1]
    channel_readings = result[2:]

    #frame = pd.DataFrame([[adc_count[0], adc_count, channel_readings[0]]],
                         #columns=['count', 'adc_count', 'reading'])
    frame = pd.DataFrame(channel_readings, columns=['reading'])
    frame.insert(0, 'adc_count', adc_count)
    frame.insert(0, 'i', i)
    frames.append(frame)
    
readings = pd.concat(frames)

In [227]:
import numpy as np
# print pd.Series(channel_readings[channel_start_i[0]::2]).describe()
# print pd.Series(channel_readings[channel_start_i[1]::2]).describe()

df_i = readings.reset_index(drop=True)
df_i['offset'] = df_i.groupby('i')['adc_count'].transform(lambda v: v & 0x01)

count_scan = np.zeros(df_i.i.unique().shape[0], dtype='uint')
count_scan[1:] = df_i.groupby('i')['adc_count'].count().cumsum()[:-1]
df_i.loc[:, 'count_exc_scan'] = count_scan[df_i['i']]
df_i['group_i'] = np.arange(df_i.shape[0]) - df_i['count_exc_scan'] 
df_i['channel_i'] = 0
df_i.loc[(df_i.group_i & 0x01 == 1) & (df_i.offset == 0), 'channel_i'] = 1
df_i.loc[(df_i.group_i & 0x01 == 0) & (df_i.offset == 1), 'channel_i'] = 1
df_i.drop('count_exc_scan', axis=1, inplace=True)
df_i.loc[(df_i.channel_i == 0) & (df_i.reading < 1000),
         ['group_i', 'channel_i', 'reading']].shape[0]

256

In [123]:
proxy.dma_available()

256

In [ ]:
import numpy as np

In [ ]:
def leading_zeros(value, bits=32):
    for i in xrange(bits, 0, -1):
        if (value >> (bits - i)) == 1:
            return i - 1
    return bits


(31 - leading_zeros(256)) << 3

In [ ]:
9 << 3

In [ ]:
import time


N = 32

# proxy.dma_start(1 << int(np.ceil(np.log2(N))))
proxy.dma_start(256)
for i in range(N): proxy.startSingleRead(teensy.A0, teensy.ADC_0)
# time.sleep(0.01)
result = proxy.adc_read()
print result.shape[0]
result

In [ ]:
result = proxy.adc_read()
result.shape[0], result.mean()

In [ ]:
proxy.start_timer(5)

In [ ]:
1 / 100e3 / 1e-6

In [ ]:
proxy.adc_period_us()

In [ ]:
import time


proxy.startContinuous(teensy.A0, teensy.ADC_0)
while proxy.dma_available() < N: time.sleep(0.001)
proxy.stopContinuous(teensy.ADC_0)
proxy.adc_read()

In [ ]:
proxy.enableInterrupts(teensy.ADC_0)

In [ ]:
proxy.adc_timestamp_us()

In [ ]:
# for i in range(128): proxy.startSingleRead(teensy.A0, teensy.ADC_0)
# proxy.startSingleRead(teensy.A0, teensy.ADC_0)


In [ ]:
from datetime import datetime, timedelta

start = datetime.now()
start_us = proxy.microseconds()
proxy.startContinuous(teensy.A0, teensy.ADC_0)
proxy.stopContinuous(teensy.ADC_0)
start + timedelta(microseconds=proxy.adc_timestamp_us() - start_us), proxy.adc_read()

In [ ]:
import time

import pandas as pd

ADC_SPEEDS = pd.Series(['ADC_VERY_LOW_SPEED',
                        'ADC_LOW_SPEED',
                        'ADC_MED_SPEED',
                        # 'ADC_HIGH_SPEED_16BITS',
                        'ADC_HIGH_SPEED',
                        'ADC_VERY_HIGH_SPEED'],
                       index=[teensy.ADC_VERY_LOW_SPEED,
                              teensy.ADC_LOW_SPEED,
                              teensy.ADC_MED_SPEED,
                              # teensy.ADC_HIGH_SPEED_16BITS,
                              teensy.ADC_HIGH_SPEED,
                              teensy.ADC_VERY_HIGH_SPEED])


def benchmark_adc(proxy):
    rows = []

    for conversion_i, conversion_speed in ADC_SPEEDS.iteritems():
        for sampling_i, sampling_speed in ADC_SPEEDS.iteritems():
            for averaging in (0, 4, 8, 16, 32):
                for resolution in (8, 10, 12):
                    if [conversion_i, sampling_i, averaging] == [5, 5, 0]:
                        continue
                    print '.', 
                    try:
                        proxy.setAveraging(averaging, teensy.ADC_0)
                        proxy.setResolution(resolution, teensy.ADC_0)
                        proxy.setConversionSpeed(conversion_i, teensy.ADC_0)
                        proxy.setSamplingSpeed(sampling_i, teensy.ADC_0)

                        proxy.startContinuous(teensy.A0, teensy.ADC_0)
                        time.sleep(0.001)
                        proxy.stopContinuous(teensy.ADC_0)
                        period_us = proxy.adc_period_us()
                    except IOError:
                        print 'Failed for %s' % [conversion_i, sampling_i, averaging,
                                                 resolution]
                        raise
                    row = [conversion_i, sampling_i, averaging,
                           resolution, period_us]
                    rows.append(row)

    df_adc_benchmarks =  pd.DataFrame(rows, columns=['conversion_i',
                                                     'sampling_i',
                                                     'averaging',
                                                     'resolution',
                                                     'period_us'])
    df_adc_benchmarks.loc[df_adc_benchmarks.period_us < 0, 'period_us'] += 1000
#     df_adc_benchmarks.set_index(['conversion_speed', 'sampling_speed',
#                                  'averaging', 'resolution'], inplace=True)
    df_adc_benchmarks['sampling_rate_Hz'] = 1e6 / df_adc_benchmarks['period_us']
    return df_adc_benchmarks

In [ ]:
df_adc_benchmarks = benchmark_adc(proxy)

In [ ]:
%matplotlib inline

In [365]:
teensy.ADC_VERY_HIGH_SPEED

5

In [ ]:
from si_prefix import si_format


df_i = (df_adc_benchmarks.loc[(df_adc_benchmarks.resolution > 8) &
                              (df_adc_benchmarks.averaging > 0) &
                              (df_adc_benchmarks.sampling_rate_Hz >= 100e3)]
        .sort_values(by=['sampling_rate_Hz', 'resolution', 'conversion_i', 'sampling_i'],
                     ascending=False))
df_i['conversion_speed'] = ADC_SPEEDS[df_i.conversion_i].values
df_i['sampling_speed'] = ADC_SPEEDS[df_i.sampling_i].values
df_i['sampling_rate'] = df_i.sampling_rate_Hz.map(si_format)
df_i[['sampling_rate', 'averaging', 'resolution', 'conversion_speed', 'sampling_speed']]

In [ ]:
df_i[['sampling_rate', 'averaging', 'resolution', 'conversion_speed', 'sampling_speed']]
axis = (df_i.set_index(['averaging', 'resolution', 'conversion_speed', 'sampling_speed'])
        ['sampling_rate_Hz']).plot(rot=90, style='o')
axis.set_yticklabels(['%sHz' % si_format(v) for v in axis.get_yticks()])
# axis.get_figure().savefig('test.pdf', bbox_inches='tight')
pass

In [ ]:
fig.savefig

In [ ]:
import time

import pandas as pd

In [ ]:
proxy.adc_period_us()

In [ ]:
proxy.digital_write(13, 0)

In [ ]:
proxy.startContinuous(teensy.A0, teensy.ADC_0)

In [ ]:
proxy.stopContinuous(teensy.ADC_0)

In [ ]:
proxy.isContinuous(teensy.ADC_0)

In [ ]:
if proxy.dma_available() > 0:
    print proxy.dma_read()
proxy.adc_buffer(), proxy.dma_available()

In [ ]:
proxy.startSingleRead(teensy.A0, teensy.ADC_0)

In [ ]:
proxy.dma_available(), proxy.dma_empty(), proxy.dma_full()

In [ ]:
proxy.dma_read()

In [ ]:
proxy.start_timer(10)

In [ ]:
%matplotlib inline
import pandas as pd

In [ ]:
from datetime import datetime

In [ ]:
proxy.setAveraging(0, -1)

In [ ]:
data = pd.DataFrame([(proxy.analogRead(0, -1), datetime.now()) for i in xrange(200)],
                    columns=['reading', 'timestamp'])
data.set_index('timestamp').plot()

In [ ]:
proxy.adc_buffer

In [ ]:
40 * proxy.benchmark_flops(10e3) / 10e3

In [ ]:
40 * proxy.benchmark_iops(10e3) / 10e3

In [ ]:
import sys
sys.path.insert(0, r'C:\Users\Christian\Documents\GitHub\ring-buffer-testbench')

In [ ]:
from ring_buffer_testbench.cRingBuffer import cRingBuffer

In [ ]:
ring_buffer = cRingBuffer()

In [ ]:
ring_buffer.elems(), ring_buffer.b_start, ring_buffer.b_end